# Project Overview: Inference and Decoding for Sequence-to-Sequence Language Translation Model

In this part of the project, my goal is to implement the inference mechanism for the trained **Sequence-to-Sequence (Seq2Seq) model**. After successfully training the model, I aim to build and utilise separate encoder and decoder models to generate translations. This phase involves reconstructing the trained model in a way that allows me to use it for decoding new, unseen input sentences.

The approach taken here leverages the trained weights and structure of the Seq2Seq model to predict translations, effectively implementing a **sampling process** using the trained encoder-decoder architecture.

## Project Objectives
The objectives in this phase include:
- Loading the trained model to extract its encoder and decoder parts separately.
- Building the inference models for both the encoder and the decoder.
- Implementing a decoding function that generates translated sentences from the input using a sampling method.
- Testing the model with sample input sentences and displaying the translated output.

## Code Breakdown and Explanation

### 1. Importing Modules and Loading the Trained Model
I start by importing relevant modules and preprocessed data variables. I also load the trained Seq2Seq model from the file `'training_model.h5'`. After loading the model, I extract the encoder’s input and output states from the layers.

In [ ]:
from preprocessing import input_features_dict, target_features_dict, reverse_input_features_dict, reverse_target_features_dict, max_decoder_seq_length, input_docs, target_docs, input_tokens, target_tokens
from training_model import encoder_inputs, decoder_inputs, encoder_states, decoder_lstm, decoder_dense, encoder_input_data, num_decoder_tokens, latent_dim

from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import numpy as np

training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]

## 2. Building the Encoder Model
Using the loaded model, I create a separate encoder model that takes the encoder input and outputs its internal states (state_h and state_c). This model will be used to encode new input sentences during inference.

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

## 3. Building the Decoder Model
Next, I define the decoder model for inference. I start by creating input placeholders for the decoder’s hidden and cell states. The decoder takes these states as inputs, along with the actual decoder inputs, to produce the predicted token and the updated states. I then define the complete decoder model for inference.

In [ ]:
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

## 4. Decoding Function for Generating Translations
Here, I define the decode_sequence function, which takes an encoded input and generates the corresponding translated sentence using the trained decoder. The function:

- Encodes the input sentence into state vectors using the encoder model.
- Initialises an empty target sequence and sets the first token to the <START> token.
- Repeatedly runs the decoder to predict the next token until an <END> token is generated or the maximum sequence length is reached.
- Accumulates the decoded tokens to build the translated sentence.

In [ ]:
def decode_sequence(test_input):
  states_value = encoder_model.predict(test_input)

  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_features_dict['<START>']] = 1.

  decoded_sentence = ''
  stop_condition = False
  while not stop_condition:
    output_tokens, hidden_state, cell_state = decoder_model.predict(
      [target_seq] + states_value)

    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = reverse_target_features_dict[sampled_token_index]
    decoded_sentence += " " + sampled_token

    if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    states_value = [hidden_state, cell_state]

  return decoded_sentence

## 5. Testing the Model with Input Sentences
I run the decoding function on a set of test input sentences to check the performance of the model. Here, I choose a range of input sentences and display their decoded outputs.

In [ ]:
for seq_index in range(100):
  test_input = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(test_input)
  print('-')
  print('Input sentence:', input_docs[seq_index])
  print('Decoded sentence:', decoded_sentence)

## Summary
In this phase of the project, I’ve set up the inference mechanism for the Seq2Seq model. I successfully:

- Extracted and built separate models for the encoder and decoder from the trained model.
- Created a decoding function that generates translations using a sampling method.
- Tested the model by decoding a batch of input sentences and displaying their translated outputs.

With this, I am now able to use the trained Seq2Seq model to generate translations for new input sentences, completing the implementation of the language translation model.